In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# !ls

--2022-02-14 11:43:46--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz’

spark-3.2.1-bin-had 100%[===================>] 287.03M   205MB/s    in 1.4s    

2022-02-14 11:43:47 (205 MB/s) - ‘spark-3.2.1-bin-hadoop3.2.tgz’ saved [300971569/300971569]



In [ ]:
!tar -xzf /content/spark-3.2.1-bin-hadoop3.2.tgz
# !ls 

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark

In [ ]:
findspark.init("/content/spark-3.2.1-bin-hadoop3.2/")

In [ ]:
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2/'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark= SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.read.load("/content/sample_data/california_housing_train.csv", format='csv', header=True)

In [ ]:
df.limit(1).withColumn("address", df.latitude).toJSON(['UTF-8']).collect()

['{"longitude":"-114.310000","latitude":"34.190000","housing_median_age":"15.000000","total_rooms":"5612.000000","total_bedrooms":"1283.000000","population":"1015.000000","households":"472.000000","median_income":"1.493600","median_house_value":"66900.000000","address":"34.190000"}']

In [ ]:
@udf(returnType=MapType(StringType(), StringType()))
def json_struct_type(cols):
  return {"longitude": cols.longitude, "latitude": cols.latitude}

In [ ]:
df.withColumn("value", json_struct_type(struct(df.longitude, df.latitude))).toJSON().collect()

In [ ]:
@udf(returnType=ArrayType(StringType()))
def json_list_type(cols):
  return [cols.longitude, cols.latitude]

In [ ]:
df.withColumn("value", json_list_type(struct(df.longitude, df.latitude))).toJSON().first()

'{"longitude":"-114.310000","latitude":"34.190000","housing_median_age":"15.000000","total_rooms":"5612.000000","total_bedrooms":"1283.000000","population":"1015.000000","households":"472.000000","median_income":"1.493600","median_house_value":"66900.000000","value":["-114.310000","34.190000"]}'

In [ ]:
schema = ArrayType( MapType(StringType(), StringType()))

@udf(returnType=schema)
def json_dict_type(cols):
  return [{"longitude": cols.longitude, "latitude": cols.latitude}, {"longitude": cols.longitude, "latitude": cols.latitude}]

In [ ]:
df.withColumn("value", json_dict_type(struct(df.longitude, df.latitude))).toJSON().first()

'{"longitude":"-114.310000","latitude":"34.190000","housing_median_age":"15.000000","total_rooms":"5612.000000","total_bedrooms":"1283.000000","population":"1015.000000","households":"472.000000","median_income":"1.493600","median_house_value":"66900.000000","value":[{"latitude":"34.190000","longitude":"-114.310000"},{"latitude":"34.190000","longitude":"-114.310000"}]}'

In [ ]:
df.withColumn("version", lit("11111")).printSchema()

root
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- housing_median_age: string (nullable = true)
 |-- total_rooms: string (nullable = true)
 |-- total_bedrooms: string (nullable = true)
 |-- population: string (nullable = true)
 |-- households: string (nullable = true)
 |-- median_income: string (nullable = true)
 |-- median_house_value: string (nullable = true)
 |-- version: string (nullable = false)

